In [ ]:
import re
import csv
import datetime

In [ ]:
from gensim import corpora
from gensim import models
from gensim import similarities
from pprint import pprint  # pretty-printer

In [ ]:
from nltk import PorterStemmer
from nltk.corpus import stopwords

In [ ]:
init_t: datetime = datetime.datetime.now()


<br>
TASK 1 <br>
Implement the following pseudocode to calculate the variable ratio_quality using the TFIDF vectors:<br>
total_goods = 0<br>
For every article (a) on topic "Food and Drink":<br>
   Obtain the top-10 most similar articles (top-10) in Corpus to a<br>
   Count how many articles in top-10 are related to topic "Food and Drink" (goods)<br>
   total_goods = total_goods + goods<br>
ratio_quality = total_goods/(num_articles_food_and_drink*10)<br>
And measure the execution times separately for the following two subprocesses: <br>
Creating the model (from the program begin to the call similarities.MatrixSimilarity(tfidf_vectors))<br>
Implementation of the pseudocode above.<br>


import our csv file and read it

In [ ]:
csv_file_path = './news.csv'
csv_full_text = ""

In [ ]:
with open(csv_file_path, newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        row_text = ' '.join(row)
        csv_full_text += row_text + '\n'
# print the text 
print(csv_full_text)

In [ ]:
porter = PorterStemmer()

remove common words and tokenize

In [ ]:
stoplist = stopwords.words('english')
texts = [
    [porter.stem(word) for word in csv_full_text.lower().split() if word not in stoplist]
    for csv_full_text in csv_full_text
]

create mapping keyword-id

In [ ]:
dictionary = corpora.Dictionary(texts)

In [ ]:
print()
print("Mapping keyword-id:")
pprint(dictionary.token2id)

In [ ]:
id2token = dict(dictionary.items())

create the vector for each doc

In [ ]:
model_bow = [dictionary.doc2bow(text) for text in texts]

create the LDA model from bow vectors

In [ ]:
lda = models.LdaModel(model_bow, num_topics=2, id2word=dictionary, random_state=30)
# random_state: forced to always obtain the same results in all the executions
lda_vectors = []
for v in model_bow:
    lda_vectors.append(lda[v])

In [ ]:
print()
print("LDA vectors for docs (in terms of topics):")
i = 0
for v in lda_vectors:
    print(v, documents[i])
    i += 1

In [ ]:
matrix_lda = similarities.MatrixSimilarity(lda_vectors)
print()
print("Matrix similarities")
print(matrix_lda)

In [ ]:
def convert(match):
    return dictionary.id2token[int(match.group(0)[1:-1])]

In [ ]:
print("LDA Topics:")
for t in lda.print_topics(num_words=30):
    print(re.sub('"[0-9]+"', convert, str(t)))

In [ ]:
end_creation_model_t: datetime = datetime.datetime.now()

In [ ]:
print()

obtain LDA vector for the following doc<br>
doc = "Human computer interaction"

In [ ]:
doc = "trees graph human"
doc_s = [porter.stem(word) for word in doc.lower().split() if word not in stoplist]

In [ ]:
vec_bow = dictionary.doc2bow(doc_s)
vec_lda = lda[vec_bow]

calculate similarities between doc and each doc of texts using lda vectors and cosine

In [ ]:
sims = matrix_lda[vec_lda]

sort similarities in descending order

In [ ]:
sims = sorted(enumerate(sims), key=lambda item: -item[1])

In [ ]:
print()
print("Given the doc: " + doc)
print("whose LDA vector is: " + str(vec_lda))
print()
print("The Similarities between this doc and the documents of the corpus are:")
for doc_position, doc_score in sims:
    print(doc_score, documents[doc_position])

In [ ]:
end_t: datetime = datetime.datetime.now()

get execution time

In [ ]:
elapsed_time_model_creation: datetime = end_creation_model_t - init_t
elapsed_time_comparison: datetime = end_t - end_creation_model_t
print()
print('Execution time model:', elapsed_time_model_creation, 'seconds')
print('Execution time comparison:', elapsed_time_comparison, 'seconds')